In [1]:
import syft as sy
import pycolab

In [2]:
# import sys
# sy.logger.add(sink=sys.stdout)

In [3]:
domain = sy.Domain(name="DM")

In [16]:
@sy.sendable(return_type="pycolab.engine.Engine")
def make_game(level=0):
    # third party
    import numpy as np
    from pycolab import ascii_art
    from pycolab import things

    HELLO_ART = [
        "                                    ",
        "  #   #  ### #    #     ###         ",
        "  #   # #    #    #    #   #        ",
        "  ##### #### #    #    #   #        ",
        "  #   # #    #    #    #   #        ",
        "  #   #  ###  ###  ###  ###         ",
        "                                    ",
        "     @   @  @@@   @@@  @    @@@@  1 ",
        "     @   @ @   @ @   @ @    @   @ 2 ",
        "     @ @ @ @   @ @@@@  @    @   @ 3 ",
        "     @ @ @ @   @ @   @ @    @   @   ",
        "      @@@   @@@  @   @  @@@ @@@@  4 ",
        "                                    ",
    ]

    class RollingDrape(things.Drape):
        """A Drape that just `np.roll`s the mask around either axis."""

        # There are four rolls to choose from: two shifts of size 1 along both axes.
        _ROLL_AXES = [0, 0, 1, 1]
        _ROLL_SHIFTS = [-1, 1, -1, 1]

        def update(self, actions, board, layers, backdrop, all_things, the_plot):
            del board, layers, backdrop, all_things  # unused

            if actions is None:
                return  # No work needed to make the first observation.
            if actions == 4:
                the_plot.terminate_episode()  # Action 4 means "quit".

            # If the player has chosen a motion action, use that action to index into
            # the set of four rolls.
            if actions < 4:
                rolled = np.roll(
                    self.curtain,  # Makes a copy, alas.
                    self._ROLL_SHIFTS[actions],
                    self._ROLL_AXES[actions],
                )
                np.copyto(self.curtain, rolled)
                the_plot.add_reward(1)  # Give ourselves a point for moving.

    class SlidingSprite(things.Sprite):
        """A Sprite that moves in diagonal directions."""

        # We have four mappings from actions to motions to choose from. The mappings
        # are arranged so that given any index i, then across all sets, the motion
        # that undoes motion i always has the same index j.
        _DX = ([-1, 1, -1, 1], [-1, 1, -1, 1], [1, -1, 1, -1], [1, -1, 1, -1])
        _DY = ([-1, 1, 1, -1], [1, -1, -1, 1], [1, -1, -1, 1], [-1, 1, 1, -1])

        def __init__(self, corner, position, character, direction_set):
            """Build a SlidingSprite.

            Args:
            corner: required argument for Sprite.
            position: required argument for Sprite.
            character: required argument for Sprite.
            direction_set: an integer in `[0,3]` that selects from any of four
                mappings from actions to (diagonal) motions.
            """
            super(SlidingSprite, self).__init__(corner, position, character)
            self._dx = self._DX[direction_set]
            self._dy = self._DY[direction_set]

        def update(self, actions, board, layers, backdrop, all_things, the_plot):
            del board, layers, backdrop, all_things, the_plot  # unused
            # Actions 0-3 are motion actions; the others we ignore.
            if actions is None or actions > 3:
                return
            new_col = (self._position.col + self._dx[actions]) % self.corner.col
            new_row = (self._position.row + self._dy[actions]) % self.corner.row
            self._position = self.Position(new_row, new_col)

    return ascii_art.ascii_art_to_game(
        HELLO_ART,
        what_lies_beneath=" ",
        sprites={
            "1": ascii_art.Partial(SlidingSprite, 0),
            "2": ascii_art.Partial(SlidingSprite, 1),
            "3": ascii_art.Partial(SlidingSprite, 2),
            "4": ascii_art.Partial(SlidingSprite, 3),
        },
        drapes={"@": RollingDrape},
        z_order="12@34",
    )

[2021-06-24T22:53:10.634421+1000][CRITICAL][logger]][10252] <class 'syft.core.store.store_memory.MemoryStore'> __delitem__ error <UID: 871ae043f3164e63aac65cf6718cf315>.


In [17]:
# helper functions for pointer resolution
def _resolve(tuple_ptr, size=1):
    resolved = []
    for i in range(size):
        resolved.append(tuple_ptr[i].resolve_pointer_type())
    return tuple(resolved)

def resolve(tuple_ptr):
    return _resolve(tuple_ptr, 3)

In [18]:
root_client = domain.get_root_client()

In [19]:
make_game.send(root_client)

Remote Secure Exec
Compiling...
... compiled!
Accepting, code bound at entry point successfully!
Give this to your DS:
ptr = client.run(
    entrypoint="make_game",
    return_type="pycolab.engine.Engine"
)


In [20]:
# # test the entrypoint locally
# game = sy.sandbox.make_game()
# print(type(game))

In [21]:
# player 1
client_1 = domain.get_client()

In [22]:
# # player 2
# client_2 = domain.get_client()

In [23]:
# client_1 == client_2

In [24]:
int_ptr = client_1.python.Int(0)

executing with args [<syft.proxy.syft.lib.python.IntPointer object at 0x141736df0>]
executing with kwargs dict_items([])
storing something <Storable: 0>
tyoe of thing <class 'syft.core.store.storeable_object.StorableObject'>
permission {<nacl.signing.VerifyKey object at 0x140155a60>: <UID: ea3737e7f4f94febb5ec5833689eef84>, <nacl.signing.VerifyKey object at 0x14115edf0>: None}


In [25]:
game_ptr = client_1.run(
    entrypoint="make_game",
    return_type="pycolab.engine.Engine",
    args=int_ptr
)

executing with args [<syft.proxy.syft.lib.python.IntPointer object at 0x141736370>]
executing with kwargs dict_items([])
storing something <Storable: <pycolab.engine.Engine object at 0x1409be790>>
tyoe of thing <class 'syft.core.store.storeable_object.StorableObject'>
permission {<nacl.signing.VerifyKey object at 0x140155a60>: <UID: ea3737e7f4f94febb5ec5833689eef84>, <nacl.signing.VerifyKey object at 0x14115edf0>: None}


/Users/madhavajay/dev/PySyft/.tox/syft.jupyter/lib/python3.9/site-packages/pycolab/ascii_art.py:318: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  art = np.vstack(np.fromstring(line, dtype=np.uint8) for line in art)


# Engine Pointer
We get back a pointer to the game engine and from here its the normal pycolab interface.

In [ ]:
game_ptr

In [ ]:
try:
    game_ptr.get()
except Exception as e:
    print("Not allowed to get the game!")

## Start Game

In [ ]:
board_ptr, reward_ptr, discount_ptr = resolve(game_ptr.its_showtime())

In [ ]:
board_ptr, reward_ptr, discount_ptr

## Run Play

In [ ]:
down = 258
up = 259
left = 260
right = 261

In [ ]:
board_ptr, reward_ptr, discount_ptr = resolve(game_ptr.play(up))

In [ ]:
board_ptr, reward_ptr, discount_ptr

In [ ]:
board_ptr.get(request_block=True, delete_obj=False)

In [ ]:
reward_ptr.get(request_block=True, delete_obj=False)

In [ ]:
discount_ptr.get(request_block=True, delete_obj=False)